In [40]:
# !pip install -U langchain-community
# !pip install unstructured
# !pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00


In [41]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import os
import json
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

In [42]:
DATA_PATH = "./data/JSON/"

def load_data():
    docs = []
    # Loop through each file in the directory
    for filename in os.listdir(DATA_PATH):
        if filename.endswith(".json"):
            file_path = os.path.join(DATA_PATH, filename)
            with open(file_path, 'r') as file:
                json_data = json.load(file)
                json_content = json.dumps(json_data, indent=2)
                doc = Document(page_content=json_content, metadata={"source": file_path})
                docs.append(doc)
    return docs
docs = load_data()
docs[0].page_content

'[\n  {\n    "summaryreportid": 3714,\n    "entityid": 3,\n    "productid": 52,\n    "totaltxncount": 9,\n    "totaltxnamt": 9.0,\n    "successfultxncount": 0,\n    "successfultxnamt": 0.0,\n    "pendingtxncount": 9,\n    "pendingtxnamt": 9.0,\n    "failedtxncount": 0,\n    "failedtxnamt": 0.0,\n    "txndate": "2023-06-09 00:00:00",\n    "servicename": "REnquiry"\n  },\n  {\n    "summaryreportid": 3715,\n    "entityid": 3,\n    "productid": 52,\n    "totaltxncount": 24,\n    "totaltxnamt": 13401.0,\n    "successfultxncount": 0,\n    "successfultxnamt": 0.0,\n    "pendingtxncount": 24,\n    "pendingtxnamt": 13401.0,\n    "failedtxncount": 0,\n    "failedtxnamt": 0.0,\n    "txndate": "2023-06-09 00:00:00",\n    "servicename": "DoRemit"\n  },\n  {\n    "summaryreportid": 3716,\n    "entityid": 3,\n    "productid": 51,\n    "totaltxncount": 10,\n    "totaltxnamt": 3244.7,\n    "successfultxncount": 3,\n    "successfultxnamt": 300.0,\n    "pendingtxncount": 1,\n    "pendingtxnamt": 2000.0,\

In [43]:
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=500,\
  length_function=len,
  add_start_index=True
)
chunks = text_splitter.split_documents(docs)
print(f"Split {len(docs)} documents into {len(chunks)} chunks")

Split 5 documents into 8964 chunks


In [44]:
doc = chunks[4]
print(doc.page_content)
print(doc.metadata)

"pendingtxncount": 1,
    "pendingtxnamt": 2000.0,
    "failedtxncount": 7,
    "failedtxnamt": 2289.4,
    "txndate": "2023-06-12 00:00:00",
    "servicename": "PAY"
  },
  {
    "summaryreportid": 3753,
    "entityid": 3,
    "productid": 52,
    "totaltxncount": 2,
    "totaltxnamt": 202.0,
    "successfultxncount": 0,
    "successfultxnamt": 0.0,
    "pendingtxncount": 2,
    "pendingtxnamt": 202.0,
    "failedtxncount": 0,
    "failedtxnamt": 0.0,
    "txndate": "2023-06-13 00:00:00",
    "servicename": "DoRemit"
  },
  {
    "summaryreportid": 3761,
    "entityid": 3,
    "productid": 52,
    "totaltxncount": 5,
    "totaltxnamt": 603.0,
    "successfultxncount": 0,
    "successfultxnamt": 0.0,
    "pendingtxncount": 5,
    "pendingtxnamt": 603.0,
    "failedtxncount": 0,
    "failedtxnamt": 0.0,
    "txndate": "2023-06-14 00:00:00",
    "servicename": "DoRemit"
  },
  {
    "summaryreportid": 3768,
    "entityid": 3,
    "productid": 52,
    "totaltxncount": 6,
{'source': './dat

In [45]:
CHROMA_PATH = "chroma"

## remove any pre-existing dir
# if os.path.exists(CHROMA_PATH):
#     print("Removing existing directory")
#     os.rmdir(CHROMA_PATH)

db = Chroma.from_documents(chunks, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY), persist_directory=CHROMA_PATH)
db.persist()
print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}")

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)